# invoice usecase

## invoice functions

In [1]:
from ipynb.fs.full.kubernetesTool import *

In [2]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

In [3]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername,user='User1'):
    return 'x509::CN=' + user + '@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


In [4]:
import json

def getHostname(peer,user):
    splits = user.split('@')
    if len(splits) > 1:
        return splits[1]
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def query(peer,callopt,user='User1'):
    return peer.query(callopt,channel_name='invoicing18',user=user)

def invoke(peer,callopt,user='User1'):
    print('invoke',callopt,user)
    return peer.invoke(callopt,channel_name='invoicing18',user=user)


def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 
    
####    
    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def transferInvoice(peer, id='asset1',name='', user='User1'):
    callopt= '{"function":"TransferInvoice","Args":' + getArgList([id,name]) + '}'
    return invoke(peer,callopt,user=user)      
    
    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
########    
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)     

def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   

  
def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer', taxInspector=''):
    name = getSubmittingClientIdentity(peer, user=user)[0]
    #print(name)
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role,taxInspector]) + '}'
    return invoke(peer,callopt)

def getRoles(peer, user='User1'):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)

## BasicSmartContract

In [5]:
class BasicSmartContract(object):
    def __init__(self,peer=None, user='User1'):
        self.peer = peer
        self.user = user
        
    def _query(self,callopt):
        return self.peer.query(callopt,channel_name='invoicing18',user=self.user)

    def _invoke(self,callopt):
        print('invoke',callopt,user)
        return self.peer.invoke(callopt,channel_name='invoicing18',user=self.user)        
        
    def getArgList(self,asset):
        if isinstance(asset, dict):
            asset = asset.values()
        return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']'         
        
        
    def getAllKeys(self):
        callopt= '{"function":"GetAllKeys","Args":[' + ']}'
        return self._query(callopt)     

    def assetExists(self, id='asset1'):
        callopt= '{"function":"AssetExists","Args":' + self.getArgList([id]) + '}'
        return self._query(callopt)

    def deleteAsset(self, asset='asset1'):
        callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
        return self._invoke(callopt)


    def readAssetTest(self, asset='asset1'):
        callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
        return self._query(callopt)

    def updateAsset(self, arglist=None):
        callopt= '{"function":"UpdateAsset","Args":' + self.getArgList(asset) + '}'
        return self._invoke(callopt)


    def getSubmittingClientIdentity(self):
        callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
        return self._query(callopt)

    def getMSPID(self, peer):
        callopt= '{"function":"GetMSPID","Args":[' + ']}'
        return self._query(callopt) 


    def getRoleTransactions(self):
        callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
        return self._query(callopt)  


    def appendRole(self, role='Buyer', taxInspector=''):
        name = self.getSubmittingClientIdentity()[0]
        callopt= '{"function":"AppendRole","Args":' + getArgList([name,role,taxInspector]) + '}'
        return self._invoke(callopt)

    def getRoles(self):
        callopt= '{"function":"GetRoles","Args":[' + ']}'
        return self._query(callopt)

    def getAllRoles(self):
        callopt= '{"function":"GetAllRoles","Args":[' + ']}'
        return self._query(callopt)  

## InvoiceSmartContract

In [6]:
class InvoiceSmartContract(BasicSmartContract):
    def __init__(self,peer=None, user='User1'):
        super(InvoiceSmartContract, self).__init__(peer=peer, user=user)
    
    def getEmptyInvoice(self):
        callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
        return self._query(callopt)     

    def createInvoice(self, arglist=''):
        callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
        return self._invoke(callopt)    

    def listInvoices(self):
        callopt= '{"function":"ListInvoices","Args":[' + ']}'
        return self._query(callopt) 

    def listInvoice(self, id='asset1'):
        callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
        return self._query(callopt)

    def transferInvoice(self, id='asset1',name=''):
        callopt= '{"function":"TransferInvoice","Args":' + getArgList([id,name]) + '}'
        return self._invoke(callopt)     



    def receivedInvoice(self, id='asset1'):
        callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
        return self._invoke(callopt)     

    def receivedOrder(self, id='asset1'):
        callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
        return self._invoke(callopt)   

    def receivedPayment(self, id='asset1', payer=''):
        callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
        return self._invoke(callopt)  

    def taxReceived(self, id='asset1'):
        callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
        return self._invoke(callopt)     

### Enroll users

### SharePoint

In [ ]:
import subprocess

class SharePoint(object):
    def __init__(self,host='10.15.130.111:80',user='scray', pwd='scray'):
        self.host = host
        self.user = user
        self.pwd  = pwd
    
    def downloadToLocal(self, local_path='./', local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, 
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)     

    def deleteRemote(self, local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, '-X', 'DELETE',
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)    

    def createDirectoryRemote(self, local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, '-X', 'MKCOL',
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)       

    def uploadFromLocal(self, local_path='./', local_name='' ,remote_path='upload'):
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, 
                                              '-T', local_path + '/' + local_name, 
                                              'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)   

    def downloadToLocalCmdList(self, local_path='./', remote_name='' ,remote_path='/'):  
        return ['curl', '--user',self.user + ':' + self.pwd, 'http://' + self.host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]    

In [8]:
import subprocess
def downloadToLocal(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
  

def deleteRemote(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, '-X', 'DELETE',
                                               'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)    

def createDirectoryRemote(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, '-X', 'MKCOL',
                                               'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)        

In [9]:
import subprocess
def downloadToLocal(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.130.111:80',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


## wallet tool

In [ ]:
import ipywidgets as widgets

class WalletTool(object):
    def __init__(self,peer=None,sharepoint=None):
        self.peer = peer
        self.sharepoint = sharepoint
    
    def uploadWalletToolToSharepoint(self):
        self.sharepoint.uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
        self.sharepoint.uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

    def installPeerWalletTool(self):
        executePeerCmd(self.peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
        #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
        executePeerCmd(self.peer.pod_name,'apk add openssl')
        executePeerCmd(self.peer.pod_name,'apk add openjdk8-jre')
        # chmod 755

    def createPeerUser(self,user=None, admin='admin',
        cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
        key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
            cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
                   '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
            return executePeerCmd(self.peer.pod_name,toCmd(cmd))

    def installWalletTool(self):
        _out = widgets.Output()
        with _out:
            self.installPeerWalletTool()
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name,  cmd)
            executePeerCmd(self.peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name,  cmd)
            executePeerCmd(self.peer.pod_name,'chmod 755 /tmp/create-user.sh')  
        return _out   

    #def installWalletTool(self,peers):
    #    for key, peer in peers.items():
    #        self.installWalletTool()

    def addUser(self,user,org,org2):
        _out = widgets.Output()
        with _out:
            strlist = ['/tmp/create-user.sh', user,org,org2 ]
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name, 'cd /tmp && ' + cmd)    
        return output,_out      
    
    def getPeerUsers(self,org):
        strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
        #print(toCmd(strlist))
        users=[]
        lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
        for line in lines:
            #users.append(line.split('@')[0])
            users.append(line)
        return users

    def getPeerOrganizations(self):
        strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
        return executePeerCmd(self.peer.pod_name,toCmd(strlist)).splitlines() 

    def get_peer_of_user(self,user=None,peers=None):
        for key,peer in peers.items():
            orgs=getPeerOrganizations(peer)
            for org in orgs:
                #print(peer.pod_name, org)
                users = getPeerUsers(peer,org)
                for ouser in users:
                    if user == ouser:
                        return peer    

In [ ]:
import ipywidgets as widgets

# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
def installWalletTool(peers):
    _out = widgets.Output()
    with _out:
        for key, peer in peers.items():
            installPeerWalletTool(peer)
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/create-user.sh')
    
def addUser(peer,user,org,org2):
    _out = widgets.Output()
    with _out:
        #strlist = ['/tmp/create-user.sh', user, peer.name + '.kubernetes.research.dev.seeburger.de']
        strlist = ['/tmp/create-user.sh', user,org,org2 ]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, 'cd /tmp && ' + cmd)    
    return output,_out    

In [ ]:
def getPeerUsers(peer,org):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
    #print(toCmd(strlist))
    users=[]
    lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
    for line in lines:
        #users.append(line.split('@')[0])
        users.append(line)
    return users

def getPeerOrganizations(peer):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
    return executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

def get_peer_of_user(user,peers):
    for key,peer in peers.items():
        orgs=getPeerOrganizations(peer)
        for org in orgs:
            #print(peer.pod_name, org)
            users = getPeerUsers(peer,org)
            for ouser in users:
                if user == ouser:
                    return peer

#### Rollen

In [ ]:
def deleteAllRoles(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('roles'):
            deleteAsset(peer,asset=key)[1]   
 
def getUsedRoles(roles):
    _all=[]
    for element in roles:
        for role in element['roles']:
            if role not in _all:
                _all.append(role)
    return _all

In [ ]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
            #result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

### create invoice

In [ ]:
## required functions // different notebook ??????

import pandas as pd

def get_invoices():
    pd.set_option('display.max_rows', 500)
    return pd.read_parquet('./invoices_facturx.parquet')

invoices = get_invoices()

def getPathValue(m,path=''):
    if(len(path)<2):
        return m
    splits = path[1:].split("/", 1)
    value=m.get(splits[0])
    #print(splits,value)
    if not isinstance(value, type(None)):
        if len(splits) > 1:
            return getPathValue(value,path='/' + splits[1])
    return value    

def assign_invoice_to_asset(_invoice,asset):
    #print(asset)

    asset['hash'] = hash(repr(sorted(_invoice.items()))) 
    monetary = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/specifiedTradeSettlementHeaderMonetarySummation/')
    asset['invoiceNumber'] = getPathValue(_invoice,'/exchangedDocument/id')['value']
    asset['netto'] = monetary['taxBasisTotalAmount'][0]['value']
    asset['tax'] = monetary['taxTotalAmount'][0]['value']
    
    try:
        asset['taxExemptionReason']  = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/applicableTradeTax')[0]['exemptionReason']['value']
    except Exception as e:
        asset['taxExemptionReason']  = ''
 
    asset['countryBuyer'] = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/postalTradeAddress/countryID')['value']
    asset['countryOrigin']   = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/postalTradeAddress/countryID')['value']
    

In [ ]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
def get_cert_cn(cert):
    return cert.split('=')[1].split(',')[0]

def get_CN_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _name + '@' + _email.split('@')[1]

def update_invoice(peers,asset=None,invoice=None,id=None,seller='Tony.Dubois',buyer='Stephanie.Hoarau'):
    assign_invoice_to_asset(invoice,asset)
    asset['ID']='asset' + str(id)
    asset['owner'] = getSubmittingClientIdentity(get_peer_of_user(seller,peers), user=seller)[0]
    asset['buyer'] = getSubmittingClientIdentity(get_peer_of_user(buyer,peers), user=buyer)[0]
    asset['taxExemptionReason'] = strip_accents(asset['taxExemptionReason'])
                
def createInvoices(invoices,peers):
    for i in range(0,20):
        try:
            invoice = invoices.iloc[i]
            seller = get_CN_of_invoice(invoice,'sellerTradeParty')
            buyer  = get_CN_of_invoice(invoice,'buyerTradeParty')
            print(i,seller,buyer,'asset'+str(i))
            seller_peer = get_peer_of_user(seller,peers)
            asset = getEmptyInvoice(seller_peer)[0]
            update_invoice(peers,asset=asset,invoice=invoice,id=i,seller=seller,buyer=buyer)
            #print(i,asset)
            createInvoice(seller_peer,getArgList(asset),user=seller)[0]
        except Exception as e:
            print(e)

def deleteAllInvoices(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('asset'):
            deleteAsset(peer,asset=key)[1]        

### Block

In [ ]:
import base64

def get_height():
    global _out
    _out = widgets.Output()
    with _out:
        strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', channel_name]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
        return json.loads(output)['height']

def uploadBlockToShare(id):
    strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', channel_name, '--block', str(id),'--publish', shared_fs_host]
    cmd = toCmd(strlist)
    output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
    
def getBlockFromShare(id):
    filename = 'invoicing-block-' + str(id) + '.block.json'
    downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/invoicing')
    output = subprocess.check_output(['cat', '/tmp/' + filename])
    return json.loads(output.decode('ascii'))           
    
def decode_block_invoice(block):
    _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
    if _transaction['value'] != None:
        return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))
    
def decode_block_transaction(block):
    _transaction = _block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
    result = []
    for arg in _transaction:
        result.append(base64.b64decode(arg).decode("utf-8"))
    return result        

def get_transaction():
    with widgets.Output():
        height=get_height()
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        return _transaction,_invoice